# **Extract data through Google API**

In [8]:
import json
import pandas as pd
import geopandas as gpd
import requests
import datetime
import matplotlib.pyplot as plt

##### **Steps**:
* Link with API
* ..

In [5]:
def get_API_key(path):
    with open(path, 'r') as f:
        return f.read().strip()

# Note: add this folder manually add the pointed path for the script to run
# For the key, ask the owner of the repository or try brute force
path_API_key = '../../data/keys/API_key.txt'

In [11]:
def get_country_codes(path):
    with open(path, 'r') as f:
        return json.load(f)


path_country_codes = '../../data/country_codes.json'
dict_country_codes = get_country_codes(path_country_codes)